# Laboratorio 6

# Analisis de Redes Sociales

### Cristina Bautista 161260
### Andy Castillo 18040
### Marco Fuentes 18188
### Abril Palencia 18198

## Problema 4

### Descarga, carga - Cristina y Andy

### Preprosecamiento - Cristina

### Analisis - Marco y Abril

### Conclusiones - Andy